In [61]:
# load packages
from sklearn import metrics,preprocessing,model_selection
from sklearn.metrics import accuracy_score
import keras
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en import English
parser = English()

import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
from gensim.models.doc2vec import TaggedDocument



from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from keras import backend as K
from sklearn.tree import DecisionTreeClassifier

In [62]:
elmo = hub.load('/Users/andrewsimon/Downloads/elmo_3')

In [63]:
main_data = pd.read_csv('/Users/andrewsimon/Desktop/Dow_dat.csv')

In [64]:
main_data['title'] = main_data['Report'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [65]:
main_data

,Level,Report,title
0,2,"On April 10th at 12:30 PM, there was an incide...","[on, april, th, at, pm, there, was, an, incide..."
1,3,"On April 8th at 9:00 PM, a fire broke out in t...","[on, april, th, at, pm, fire, broke, out, in, ..."
2,4,"On April 6th at 2:45 AM, there was an incident...","[on, april, th, at, am, there, was, an, incide..."
3,1,"On April 5th at 7:15 PM, there was an unplanne...","[on, april, th, at, pm, there, was, an, unplan..."
4,5,"On April 2nd at 11:30 AM, there was a minor in...","[on, april, nd, at, am, there, was, minor, inc..."
...,...,...,...
295,3,"At 9:45 AM, a worker reported a small leak of ...","[at, am, worker, reported, small, leak, of, ch..."
296,5,"At 2:30 PM, a worker reported a small spill of...","[at, pm, worker, reported, small, spill, of, c..."
297,4,"At 6:15 AM, a worker reported an unusual odor ...","[at, am, worker, reported, an, unusual, odor, ..."
298,2,"At 11:00 AM, a worker reported a small fire in...","[at, am, worker, reported, small, fire, in, th..."


In [66]:
embeddings = []

for sentence in main_data['Report']:
    tensor = tf.constant([sentence])
    embedding = elmo.signatures['default'](tensor)['elmo']
    embedding = tf.reduce_mean(embedding, 1)
    embeddings.append(embedding)


In [67]:
main_data.drop(columns=['Report'])

,Level,title
0,2,"[on, april, th, at, pm, there, was, an, incide..."
1,3,"[on, april, th, at, pm, fire, broke, out, in, ..."
2,4,"[on, april, th, at, am, there, was, an, incide..."
3,1,"[on, april, th, at, pm, there, was, an, unplan..."
4,5,"[on, april, nd, at, am, there, was, minor, inc..."
...,...,...
295,3,"[at, am, worker, reported, small, leak, of, ch..."
296,5,"[at, pm, worker, reported, small, spill, of, c..."
297,4,"[at, am, worker, reported, an, unusual, odor, ..."
298,2,"[at, am, worker, reported, small, fire, in, th..."


In [68]:
X_train, X_test, y_train, y_test = train_test_split(embeddings,
                                                    main_data['Level'], test_size=0.2, random_state=42)

In [69]:
X_train = np.array(X_train)
X_train = X_train.reshape(240, 1024)

X_test = np.array(X_test)
X_test = X_test.reshape(60, 1024)

X_train.shape

(240, 1024)

In [70]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression( max_iter = 100000)
classifier.fit(X_train_scaled, y_train)
predictions_LR = classifier.predict(X_test_scaled)
score = classifier.score(X_test_scaled, y_test)
score

NameError: name 'X_train_scaled' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

len(y_test)
cm = confusion_matrix(y_test, predictions_LR)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = [1,2,3,4,5])
disp.plot()

In [ ]:
clf_decision_tree = DecisionTreeClassifier()
clf_decision_tree.fit(X_train, y_train)
predictions_DT = clf_decision_tree.predict(X_test)
acc_score = accuracy_score(y_test, predictions_DT)
acc_score

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

len(y_test)
cm = confusion_matrix(y_test, predictions_DT)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = [1,2,3,4,5])
disp.plot()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1100, random_state=11).fit(X_train_scaled, y_train)
predictions = rf.predict(X_test_scaled)
score = rf.score(X_test_scaled, y_test)
score

In [ ]:
X_train.shape

In [ ]:
import tensorflow_decision_forests as tfdf

In [ ]:
X_train_new = []


for i in X_train:
    sub_X_train_new = []
    for j in i:
        sub_X_train_new.append(j)
    X_train_new.append(sub_X_train_new)


X_train_new = np.array(X_train_new)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

2/2 - 0s - loss: 1.1934 - accuracy: 0.7667 - 65ms/epoch - 33ms/step


In [91]:
import tensorflow as tf

model = tf.keras.Sequential([
     tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='sigmoid'),
    tf.keras.layers.Dense(6, activation='sigmoid'),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, verbose=2)



Epoch 1/100
8/8 - 0s - loss: 1.7319 - accuracy: 0.2083 - 161ms/epoch - 20ms/step
Epoch 2/100
8/8 - 0s - loss: 1.5286 - accuracy: 0.3667 - 16ms/epoch - 2ms/step
Epoch 3/100
8/8 - 0s - loss: 1.3774 - accuracy: 0.4417 - 17ms/epoch - 2ms/step
Epoch 4/100
8/8 - 0s - loss: 1.2226 - accuracy: 0.4958 - 16ms/epoch - 2ms/step
Epoch 5/100
8/8 - 0s - loss: 1.0902 - accuracy: 0.5458 - 18ms/epoch - 2ms/step
Epoch 6/100
8/8 - 0s - loss: 1.0070 - accuracy: 0.5500 - 16ms/epoch - 2ms/step
Epoch 7/100
8/8 - 0s - loss: 0.9605 - accuracy: 0.5792 - 14ms/epoch - 2ms/step
Epoch 8/100
8/8 - 0s - loss: 0.9041 - accuracy: 0.6083 - 19ms/epoch - 2ms/step
Epoch 9/100
8/8 - 0s - loss: 0.8795 - accuracy: 0.6125 - 27ms/epoch - 3ms/step
Epoch 10/100
8/8 - 0s - loss: 0.8425 - accuracy: 0.6500 - 33ms/epoch - 4ms/step
Epoch 11/100
8/8 - 0s - loss: 0.8103 - accuracy: 0.6417 - 24ms/epoch - 3ms/step
Epoch 12/100
8/8 - 0s - loss: 0.8420 - accuracy: 0.5875 - 17ms/epoch - 2ms/step
Epoch 13/100
8/8 - 0s - loss: 0.8050 - accuracy

In [93]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

2/2 - 0s - loss: 0.7877 - accuracy: 0.7333 - 16ms/epoch - 8ms/step


In [94]:
preds = model.predict(X_test)

2/2 [==============================] - 0s 2ms/step


In [95]:
len(preds[0])

6

In [96]:
predsls

array([[2.59356834e-02, 9.99866962e-01, 2.20301747e-02, 8.21515977e-01,
        4.48623627e-01, 4.70394781e-03],
       [1.08756116e-02, 4.96712565e-01, 7.51141191e-01, 9.85620558e-01,
        4.50951457e-01, 8.21731519e-03],
       [1.64520573e-02, 2.62543801e-02, 7.19634473e-01, 7.93732345e-01,
        6.21681325e-02, 9.87076044e-01],
       [2.57021822e-02, 5.34225702e-02, 1.08930141e-01, 6.01479828e-01,
        1.63410474e-02, 9.99921799e-01],
       [1.12405065e-02, 2.75147315e-02, 9.80063498e-01, 8.78454864e-01,
        8.89253199e-01, 6.01646909e-03],
       [1.36131439e-02, 1.22215636e-01, 8.88317823e-01, 7.43446112e-01,
        9.85991597e-01, 1.52226794e-03],
       [1.19785974e-02, 7.57989138e-02, 9.40314353e-01, 6.37489736e-01,
        9.79413927e-01, 4.37396392e-03],
       [1.45110376e-02, 7.13098422e-02, 5.18852830e-01, 8.13719511e-01,
        9.89077985e-01, 1.24637550e-02],
       [1.09913731e-02, 9.08644944e-02, 8.91994476e-01, 9.55794215e-01,
        8.54385436e-01, 

In [76]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = [1,2,3,4,5])
disp.plot()

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

len(y_test)
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = [1,2,3,4,5])
disp.plot()

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)
SVM_predict = clf.predict(X_test)
score = clf.score(X_test, y_test)
score

In [ ]:
len(y_test)
cm = confusion_matrix(y_test, SVM_predict)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = [1,2,3,4,5])
disp.plot()